<a href="https://colab.research.google.com/github/Asif2804/energy-management-algorithm/blob/main/XAI_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Literature to compare XAI models: [link text](https://arxiv.org/html/2503.04261v1?)

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import tracemalloc
import random

from collections import defaultdict
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, r2_score
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

#!pip install lime
#!pip install dice-ml


from lime.lime_tabular import LimeTabularExplainer
import shap
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
import dice_ml
from dice_ml import Dice
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree


ModuleNotFoundError: No module named 'lime'

In [ ]:
#Loading Data
df = pd.read_csv('/content/heart.csv')

**Attribute information:**
1. age
2. sex
3. chest pain type
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results
8. maximum heart rate achieved
9. exercise induced angina - (Wether the patient has chestpain durng Exercise)
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment ()
12. number of major vessels (0-3) colored by flourosopy
13. thal: 0 = normal; 1 = fixed defect; 2 = reversable defect - (A type of blood disorder tested during heart evaluation)

# Exploratory Data Analysis

In [ ]:
#Check correlations with target
correlations = df.corr(numeric_only=True)['target'].sort_values(ascending=False)
print("\nCorrelations with target:\n", correlations)

#Check target class distribution
print("\nTarget class distribution:\n", df['target'].value_counts(normalize=True))

#Plot class distribution
df['target'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title("Class Distribution Before Duplicate Removal")
plt.xlabel("Target (0 = No Disease, 1 = Heart Disease)")
plt.ylabel("Count")
plt.show()


In [ ]:
#Drop duplicate rows
df = df.drop_duplicates()
print(f"\nData shape after dropping duplicates: {df.shape}")

#Class distribution after duplicate removal
df['target'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title("Class Distribution After Duplicate Removal")
plt.xlabel("Target (0 = No Disease, 1 = Heart Disease)")
plt.ylabel("Count")
plt.show()

NameError: name 'df' is not defined

In [ ]:
#Separate features (X) and target label (y)
X = df.drop("target", axis=1)
y = df["target"]

#Treat all features as numeric for simplicity
numerical_cols = X.columns.tolist()  #all columns are numeric

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

#Single transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols)
    ])


for run in range(5):  #number of runs
    random_seed = np.random.randint(0, 61)

#Train test split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state= random_seed )


# Training models

F1 - bELOW 0.75 is not admissable

In [ ]:
#Print test set shape
print(X_test_raw.shape)  #How many rows in the test set?

#4 ML models
models = {
    "Random Forest": RandomForestClassifier(
        n_estimators=100, max_depth=5, random_state=42  #No of trees, depth of each tree,
    ),
    "KNN": KNeighborsClassifier(
        n_neighbors=5, weights='uniform', metric='minkowski' #No. of neighbors used, equal weight across neighbors, default distance
    ),
    "XGBoost": XGBClassifier(
        n_estimators=200, max_depth=5, learning_rate=0.05, eval_metric='logloss', random_state=42 #shrinking stepsize, evaluation metric
    ),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(32, 16), max_iter=500, alpha=0.001, random_state=42 #2 hidden layers, max num of training, regularization
    )
}

pipelines = {
    name: Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    for name, model in models.items()
}

results = {} #Store trained pipelines and metrics

for name, pipeline in pipelines.items():
    pipeline.fit(X_train_raw, y_train) #Fit pipeline on data
    y_pred = pipeline.predict(X_test_raw) #Predict labels on test data

    results[name] = {                       #Store trained pipeline and metrics
        "pipeline": pipeline,
        "accuracy": accuracy_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    }

# Print metrics
for name, metrics in results.items():
    print(f"\n--- {name} ---")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")


In [ ]:
#Convert results to a DataFrame
metrics_df = pd.DataFrame(results).T  #transpose so models are rows

#Plot grouped bar chart
metrics_df.plot(kind="bar", figsize=(10, 6))
plt.title("Model Performance Comparison")
plt.ylabel("Score")
plt.ylim(0, 1)  #since metrics are between 0 and 1
plt.xticks(rotation=0)
plt.legend(title="Metric")
plt.show()

NameError: name 'results' is not defined

# Lime for Random Forest (local explanation)

In [ ]:
print(X_test_raw.shape)  #How many rows in the test set?

#Extract preprocessor and trained model
rf_pipeline = results["Random Forest"]["pipeline"]         #Take trained pipeline from above
preprocessor = rf_pipeline.named_steps["preprocessor"]     #Accesses to pull out just the data transformation part
rf_model = rf_pipeline.named_steps["classifier"]           #Accessing to pull out just the ML model

#Preprocess full training data for LIME background
X_train_processed = preprocessor.transform(X_train_raw)

#Create LIME explainer
explainer = LimeTabularExplainer(
    training_data=X_train_processed, #The processed training data
    feature_names=preprocessor.get_feature_names_out(), #post pre-processing
    class_names=['No Disease', 'Heart Disease'], #Target lable
    mode='classification',
    discretize_continuous=True
)

#One test sample patient
sample_index = 44
X_one_raw = X_test_raw.iloc[[sample_index]] #Data before pre-processing
X_one_transformed = preprocessor.transform(X_one_raw) #Preprocessed Data

#Wrap predict_proba() so LIME can call it
def rf_predict_proba(x):
    return rf_model.predict_proba(x)

#LIME explanation
lime_exp = explainer.explain_instance(
    data_row=X_one_transformed[0], # processed features for your one patient
    predict_fn=rf_predict_proba, #The function LIME will call to get model predictions
    num_features=7

)

r2 = lime_exp.score
print(f"LIME surrogate R² fidelity: {r2:.4f}")

#Visualizing
lime_exp.show_in_notebook()

LIME graph is explaining the model predicted a 61% probability of Heart Disease for patient 43.

The most influcential features are:
*   Thalassemia
*   Slope of the peak exercise ST segmen
*   Exercise-Induced Angina

Since the fidelity is low (0.26), the linear approximation doesnt capture the full complexity of the model here.



# Lime for K-nearest Neighbor (local explanation)

In [ ]:
print(X_test_raw.shape)  #How many rows in the test set?

#Extract trained KNN pipeline
knn_pipeline = results["KNN"]["pipeline"]  #Taking the pipeline for KNN
preprocessor = knn_pipeline.named_steps["preprocessor"]
knn_model = knn_pipeline.named_steps["classifier"]

# Preprocess training data for background
X_train_processed = preprocessor.transform(X_train_raw)

# Create LIME explainer
lime_explainer = LimeTabularExplainer(
    training_data=X_train_processed,          # Processed training data
    feature_names=preprocessor.get_feature_names_out(), #Names after processing
    class_names=['No Disease', 'Heart Disease'], #Target value
    mode='classification',                       #classification problem
    discretize_continuous=True
)

#test sample
sample_index = 36
X_sample_raw = X_test_raw.iloc[[sample_index]] #raw data before the pre-processing
X_sample_transformed = preprocessor.transform(X_sample_raw) # preprocessed data

def knn_predict(X):
    return knn_model.predict_proba(X)

#LIME explanation
lime_exp = lime_explainer.explain_instance(
    data_row=X_sample_transformed[0], #processed features for this patient
    predict_fn=knn_predict, #function LIME calls to get predictions
    num_features=7 # top 7 fearues
)
r2 = lime_exp.score
print(f"LIME surrogate R² fidelity: {r2:.4f}")

#Show LIME explanation
lime_exp.show_in_notebook()

LIME graph is explaining the model predicted a 80% probability of Heart Disease for patient 43.

The most influcential features are:
*   Thalassemia
*   Exercise-Induced Angina
*   Slope of the peak exercise ST segmen

Since the fidelity is again low (0.41), the linear approximation does not approximate the Random Forests local behavior very well for this patient.


# Lime for XGBoost (local explanation)

In [ ]:
#XGBoost pipeline
xgb_pipeline = results["XGBoost"]["pipeline"]

#Prepare the training data
preprocessor = xgb_pipeline.named_steps["preprocessor"]
X_train_processed = preprocessor.transform(X_train_raw)
feature_names = preprocessor.get_feature_names_out()

#Set up the LIME explainer
explainer = LimeTabularExplainer(
    training_data=X_train_processed,
    feature_names=feature_names,
    class_names=["No Disease", "Heart Disease"],  #Match class names
    mode="classification",
    discretize_continuous=True
)

#Test instance to explain
instance_idx = 22
instance = X_test_raw.iloc[[instance_idx]]  #DataFrame slice for correct columns

#Preprocess the instance
instance_processed = preprocessor.transform(instance)

#Ensures input is always in the right format
def xgb_predict_proba_wrapper(X):
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=X_train_raw.columns)
    return xgb_pipeline.predict_proba(X)

#Generate the LIME explanation
exp = explainer.explain_instance(
    data_row=instance_processed[0],
    predict_fn=xgb_predict_proba_wrapper,
    num_features=7
)

#visiualize
exp.show_in_notebook(show_table=True, show_all=False)


# Lime for MPL Classifire (local explaination)

In [ ]:
mlp_pipeline = results["MLP"]["pipeline"]

#Preparetraining data
preprocessor = mlp_pipeline.named_steps["preprocessor"]
X_train_processed = preprocessor.transform(X_train_raw)
feature_names = preprocessor.get_feature_names_out()

#Initialize LIME explainer
explainer_mlp = LimeTabularExplainer(
    training_data=X_train_processed,
    feature_names=feature_names,
    class_names=["No Disease", "Disease"],
    mode="classification",
    discretize_continuous=True
)

#Choose a test instance
instance_idx = 36
instance = X_test_raw.iloc[[instance_idx]] #Pass a DataFrame slice

#Preprocess the instance for the MLP pipeline
instance_processed = preprocessor.transform(instance)


#predict_fn that handles both DataFrame and NumPy array inputs
def mlp_predict_proba_wrapper(X):
    #Ensure input is a DataFrame with correct column names
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=X_train_raw.columns) #Use original column names
    return mlp_pipeline.predict_proba(X)


#Generate the explanation
exp_mlp = explainer_mlp.explain_instance(
    data_row=instance_processed[0], #Use the processed instance data row
    predict_fn=mlp_predict_proba_wrapper, #Use the wrapper function
    num_features= 7 # top 7 features
)

#Visualize
exp_mlp.show_in_notebook(show_table=True, show_all=False)


# SHAP for Random Forest

In [ ]:
##################### Global #######################

#Get RF pipeline from results
rf_pipeline = results['Random Forest']['pipeline']
#Get preprocessor and classifier
preprocessor = rf_pipeline.named_steps['preprocessor']
rf_model = rf_pipeline.named_steps['classifier']

X_test_transformed = preprocessor.transform(X_test_raw) #Use raw test set to get full transformed data
feature_names = preprocessor.get_feature_names_out() #feature names

#TreeExplainer for the RF
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test_transformed) #Pass full test set

#For class 1 (Heart Disease)
shap_values_class1 = shap_values[:, :, 1]  #Switch between class 1/2

#
# This should now work
shap.summary_plot(
    shap_values_class1,
    X_test_transformed,
    feature_names=feature_names
)


#Get models original probability predictions for class 1
y_pred_full = rf_model.predict_proba(X_test_transformed)[:, 1]





NameError: name 'results' is not defined

SHAP global summary plot is explaining which features most influence the Random Forest models predictions for Heart Disease across all patients.

The most influential features are:
* Chest Pain Type (num__cp)
* Number of Major Vessels (num__ca)
* Thalassemia (num__thal)
* Exercise-Induced Angina (num__exang)
* Maximum Heart Rate Achieved (num__thalach)

Since the fidelity is very high (0.99), the top features identified by SHAP capture nearly all of the Random Forests behavior, meaning this explanation closely matches the models predictions.

In [ ]:
####################### Local SHAP Explanation ############################
print(X_test_raw.shape)  #How many rows in the test set?

sample_index = 43  #patient
#Slice one row
X_sample = X_test_transformed[sample_index:sample_index+1]

#Get SHAP values for this sample
shap_values_sample = explainer.shap_values(X_sample)

#The SHAP values for class 1 (Heart Disease)
shap_values_sample_class1 = shap_values_sample[:, :, 1]

#Convert to 1D array for waterfall plot
shap_values_flat = shap_values_sample_class1.flatten()

#Get expected value for class 1
expected_value = explainer.expected_value[1]

#input features
X_dense = X_sample.toarray()[0] if hasattr(X_sample, "toarray") else X_sample[0]

# Plot waterfall
shap.plots._waterfall.waterfall_legacy(
    expected_value=expected_value,
    shap_values=shap_values_flat,
    feature_names=feature_names,
    features=X_dense,
    max_display=13  # Show all features
)


SHAP local explanation is explaining the model predicted a 61% probability of Heart Disease for patient 43.

The most influential features are:

* Chest Pain Type (num__cp)
* Exercise-Induced Angina (num__exang)
* Number of Major Vessels (num__ca)
* Thalassemia (num__thal)
* Maximum Heart Rate Achieved (num__thalach)

Since SHAP calculates contributions directly from the model without relying on a linear surrogate, the explanation Should be accurate for the Random Forests prediction for this patient.





# SHAP for K-Nearest Neighbor

In [ ]:
##################### Global #######################
USE_SUBSET =  False     #False/True
SUBSET_SIZE = 2      #Samples

#100 random training samples
X_background = preprocessor.transform(X_train_raw.sample(100, random_state=42))
X_background = X_background.toarray() if hasattr(X_background, "toarray") else X_background

#Transform test set with the same preprocessor
X_test_transformed = preprocessor.transform(X_test_raw)
X_test_dense = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed

#Use subset or full test set
X_test_subset = X_test_dense[:SUBSET_SIZE] if USE_SUBSET else X_test_dense

#Wrapper so SHAP only explains probability for class 1 (Heart disease)
def predict_class1(X):
    return knn_model.predict_proba(X)[:, 1]

#KernalSHAP created
explainer = shap.KernelExplainer(predict_class1, X_background)

#Compute SHAP values for the subset
shap_vals_raw = explainer.shap_values(X_test_subset)

shap_vals_class1 = np.array(shap_vals_raw)

#Fix the shape issues
if shap_vals_class1.shape[::-1] == X_test_subset.shape:
    shap_vals_class1 = shap_vals_class1.T
elif shap_vals_class1.shape != X_test_subset.shape:
    raise ValueError(f"Unexpected SHAP shape {shap_vals_class1.shape}, expected {X_test_subset.shape}")


#SHAP Summary Plot
expl_all = shap.Explanation(
    values=shap_vals_class1,
    base_values=explainer.expected_value,
    data=X_test_subset,
    feature_names=preprocessor.get_feature_names_out()
)
shap.plots.beeswarm(expl_all, max_display=50)

SHAP global explanation is explaining which features most influence the KNN models predictions for Heart Disease across all patients.

The most influential features are:

* Thalassemia (num__thal)
* Sex (num__sex)
* Exercise-Induced Angina (num__exang)
* Chest Pain Type (num__cp)
* Maximum Heart Rate Achieved (num__thalach)

The top features account for most of the variation in the models predictions, as shown by a SHAP fidelity score of R² = 0.8851 when only the top 10 features are used.










In [ ]:
####################### Local SHAP Explanation ############################
print(X_test_raw.shape)  #How many rows in the test set?

sample_index = 43
X_one_raw = X_test_raw.iloc[[sample_index]]
X_one_transformed = preprocessor.transform(X_one_raw)

#KernelExplainer needs a background dataset
background = shap.sample(X_train_processed, 50, random_state=42)

#Use predict_proba for SHAP
def knn_predict(x):
    return knn_model.predict_proba(x)  #make sure knn_model is your fitted KNN classifier

#KernelExplainer
explainer_knn = shap.KernelExplainer(knn_predict, background)

#Compute SHAP values for this sample (for both classes)
shap_values = explainer_knn.shap_values(X_one_transformed)

#for class 1 (Heart Disease)
shap_values_class1 = shap_values[0, :, 1]  #SHAP values for patient 100, class 1

#Create SHAP Explanation object for waterfall plot
expl_knn = shap.Explanation(
    values=shap_values_class1,
    base_values=explainer_knn.expected_value[1],
    data=X_one_transformed.toarray()[0] if hasattr(X_one_transformed, "toarray") else X_one_transformed[0],
    feature_names=preprocessor.get_feature_names_out(),
)

#Plot waterfall
shap.plots.waterfall(expl_knn, max_display=13)

SHAP local explanation is explaining the model predicted an 80% probability of Heart Disease for patient 43.

The most influential features are:

* Chest Pain Type (num__cp)
* Thalassemia (num__thal)
* Sex (num__sex)
* Maximum Heart Rate Achieved (num__thalach)
* Serum Cholesterol (num__chol)

Since SHAP calculates contributions directly from the KNN model without relying on a surrogate, the explanation accurately reflects the models prediction for this patient.

# SHAP for XGBoost

In [ ]:
#Get trained XGBoost pipeline and the classifier
xgb_pipeline = results["XGBoost"]["pipeline"]
preprocessor = xgb_pipeline.named_steps["preprocessor"]
xgb_model = xgb_pipeline.named_steps["classifier"]  #the actual XGBClassifier

#Preprocess training and test sets
X_background = preprocessor.transform(X_train_raw.sample(100, random_state=42))
X_background = X_background.toarray() if hasattr(X_background, "toarray") else X_background

X_test_transformed = preprocessor.transform(X_test_raw)
X_test_dense = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed

#Optionally use a subset of test data for speed
USE_SUBSET = False
SUBSET_SIZE = 2
X_test_subset = X_test_dense[:SUBSET_SIZE] if USE_SUBSET else X_test_dense

#Create SHAP TreeExplainer
explainer = shap.TreeExplainer(xgb_model)

#Compute SHAP values for both classes
shap_values = explainer.shap_values(X_test_subset)
shap_vals_class1 = shap_values[1] if isinstance(shap_values, list) else shap_values  #binary models

#beeswarm plot
expl_all = shap.Explanation(
    values=shap_vals_class1,
    base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, (list, np.ndarray)) else explainer.expected_value,
    data=X_test_subset,
    feature_names=preprocessor.get_feature_names_out()
)

shap.plots.beeswarm(expl_all, max_display=50)



NameError: name 'results' is not defined

In [ ]:
##################### Local #######################
xgb_pipeline = results["XGBoost"]["pipeline"]
preprocessor = xgb_pipeline.named_steps["preprocessor"]
xgb_model = xgb_pipeline.named_steps["classifier"]

X_test_transformed = preprocessor.transform(X_test_raw)
X_test_dense = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed

#Create a SHAP TreeExplainer for xgb
explainer = shap.TreeExplainer(xgb_model)

#patient
patient_idx = 43  #Change this index for other patients
x_patient = X_test_dense[patient_idx:patient_idx+1]  #Keep as 2D

#Compute SHAP values for this patient
shap_values = explainer.shap_values(x_patient)
shap_vals_class1 = shap_values[1] if isinstance(shap_values, list) else shap_values

#Build a SHAP Explanation object for plotting
expl_patient = shap.Explanation(
    values=shap_vals_class1[0],
    base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, (list, np.ndarray)) else explainer.expected_value,
    data=x_patient[0],
    feature_names=preprocessor.get_feature_names_out()
)

#Visualize as a waterfall plot (local explanation)
shap.plots.waterfall(expl_patient, max_display=10)


# SHAP for MLP

In [ ]:
##################### Global #######################

USE_SUBSET = False
SUBSET_SIZE = 2

#MLP pipeline and classifier
mlp_pipeline = results["MLP"]["pipeline"]
preprocessor = mlp_pipeline.named_steps["preprocessor"]
mlp_model = mlp_pipeline.named_steps["classifier"]

X_background = preprocessor.transform(X_train_raw.sample(100, random_state=42))
X_background = X_background.toarray() if hasattr(X_background, "toarray") else X_background

X_test_transformed = preprocessor.transform(X_test_raw)
X_test_dense = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed

#Use subset or full test set
X_test_subset = X_test_dense[:SUBSET_SIZE] if USE_SUBSET else X_test_dense

#Wrap
def predict_class1(X):
    return mlp_model.predict_proba(X)[:, 1]

#Create a KernelExplainer
explainer = shap.KernelExplainer(predict_class1, X_background)

#Compute SHAP values for the subset
shap_vals_raw = explainer.shap_values(X_test_subset)

shap_vals_class1 = np.array(shap_vals_raw)

#Fix shape mismatch
if shap_vals_class1.shape[::-1] == X_test_subset.shape:
    shap_vals_class1 = shap_vals_class1.T
elif shap_vals_class1.shape != X_test_subset.shape:
    raise ValueError(f"Unexpected SHAP shape {shap_vals_class1.shape}, expected {X_test_subset.shape}")


#Create a SHAP Explanation
expl_all = shap.Explanation(
    values=shap_vals_class1,
    base_values=explainer.expected_value,
    data=X_test_subset,
    feature_names=preprocessor.get_feature_names_out()
)

#beeswarm plot
shap.plots.beeswarm(expl_all, max_display=50)


In [ ]:
######################## LOCAL #######################
#MLP pipeline and components
mlp_pipeline = results["MLP"]["pipeline"]
preprocessor = mlp_pipeline.named_steps["preprocessor"]
mlp_model = mlp_pipeline.named_steps["classifier"]

X_background = preprocessor.transform(X_train_raw.sample(100, random_state=42))
X_background = X_background.toarray() if hasattr(X_background, "toarray") else X_background

X_test_transformed = preprocessor.transform(X_test_raw)
X_test_dense = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed

#Select one patient to explain
patient_idx = 0
x_patient = X_test_dense[patient_idx:patient_idx+1]

#probability for Heart Disease (class 1)
def mlp_predict_class1(X):
    return mlp_model.predict_proba(X)[:, 1]

#Create KernelExplainer
explainer = shap.KernelExplainer(mlp_predict_class1, X_background)

#Compute SHAP values for this patient
shap_vals = explainer.shap_values(x_patient)
shap_vals_class1 = np.array(shap_vals)  #convert to array if list

#Ensure the shape matches the input
if shap_vals_class1.ndim > 1 and shap_vals_class1.shape[0] != x_patient.shape[0]:
    shap_vals_class1 = shap_vals_class1.T

#Explainer
expl_patient = shap.Explanation(
    values=shap_vals_class1[0],
    base_values=explainer.expected_value,
    data=x_patient[0],
    feature_names=preprocessor.get_feature_names_out()
)


#local waterfall plot
shap.plots.waterfall(expl_patient, max_display=10)


# PFI Setup

In [ ]:
########################## PFI Model ################################
def run_pfi(model,          #Trained model
            X_test, y_test, #Preprocessed test data
            feature_names,  #List or array of feature names
            model_name="Model", #Name for labeling the plot
            metric='roc_auc', #Performance metric used
            repeats=10 #No of shuffles per feature.
            ):


    result = permutation_importance(model, X_test, y_test,
                                     n_repeats=repeats,  #Randomly shuffles that feature repeats times for each feature
                                     random_state=42,
                                     scoring=metric) #Measures how much the models performance drops, larger performance drops = feature is more important

    #Sorts the features by average importance
    sorted_idx = np.argsort(result.importances_mean)[::-1]

    #Print table of top features
    for i in sorted_idx:
     print(f"{feature_names[i]}: {result.importances_mean[i]:.4f} ± {result.importances_std[i]:.4f}")

    #Bar chart of importance
    plt.figure(figsize=(8, 5))
    plt.barh(np.array(feature_names)[sorted_idx], result.importances_mean[sorted_idx])
    plt.xlabel(f"Permutation Importance ({metric})")
    plt.title(f"Global Feature Importance - {model_name}")
    plt.gca().invert_yaxis()
    plt.show()

    return result


# PFI for Random Forest

In [ ]:
feature_names = preprocessor.get_feature_names_out()

print("Permutation Feature Importance for RandomForest")
rf_pfi = run_pfi(rf_model, X_test_transformed, y_test, feature_names, model_name="RandomForest")

NameError: name 'preprocessor' is not defined

# PFI for K-nearest Neighboor

In [ ]:
print("Permutation Feature Importance for KNN")
knn_pfi = run_pfi(knn_model, X_test_transformed, y_test, feature_names, model_name="KNN")


# PFI for XG-Boost

In [ ]:
print("Permutation Feature Importance for XGBoost")
rf_pfi = run_pfi(xgb_model, X_test_transformed, y_test, feature_names, model_name="XGBoost")

Permutation Feature Importance for XGBoost


NameError: name 'run_pfi' is not defined

# PFI for Multi-Layer Perceptron

In [ ]:
print("Permutation Feature Importance for Multi-Layer Perception")
rf_pfi = run_pfi(mlp_model, X_test_transformed, y_test, feature_names, model_name="MLP")

# PDP/ICE Setup

In [ ]:
def plot_pdp_ice_all_numeric_safe(pipeline, X_train, model_name, features):
    #Get feature names from training data
    raw_feature_names = list(X_train.columns)

    for feature in features:            #Skip if feature is not in the dataset
        if feature not in raw_feature_names:
            print(f"Feature '{feature}' not found in X_train columns!")
            continue

        feature_idx = raw_feature_names.index(feature)
        unique_vals = np.unique(X_train[feature])

        print(f"Generating PDP for '{feature}' ({model_name})...")

        if len(unique_vals) <= 4:
            #PDP for binary/categorical features
            probs = []
            for val in unique_vals:
                X_copy = X_train.copy()
                X_copy[feature] = val
                preds = pipeline.predict_proba(X_copy)[:, 1]  #probability for class 1
                probs.append(np.mean(preds))

            plt.bar([str(v) for v in unique_vals], probs)
            plt.xlabel(feature)
            plt.ylabel("Predicted Probability")
            plt.title(f"{model_name} - PDP for {feature} (categorical/binary)")
            plt.show()

        else:
            #PDP an ICE for continuous features
            PartialDependenceDisplay.from_estimator(
                pipeline,
                X_train,
                [feature_idx],
                kind='both',                #PDP + ICE
                subsample=50,
                random_state=42,
                response_method='predict_proba'
            )
            plt.title(f"{model_name} - PDP + ICE for {feature} (continuous)")
            plt.show()

# PDP/ICE for RandomForest

In [ ]:
top_7_features = ['cp', 'chol', 'age', 'ca', 'thal', 'exang', 'thalach']

#For Random Forest
plot_pdp_ice_all_numeric_safe(results["Random Forest"]["pipeline"],
                              X_train_raw,
                              "Random Forest",
                              top_7_features)

NameError: name 'plot_pdp_ice_all_numeric_safe' is not defined

# sex
The model predicts a higher probability of heart disease for Females(~0.59)
compared to Males (~0.51). The model associates Females with higher average risk.

# exang
The model shows, given the rest of the features, patients who do not experience exercise-induced angina tend to have a slightly higher predicted risk of heart disease than those who do.

# slope
Downsloping (slope=2) is associated with the highest predicted probability (~0.57).

Flat (slope=1) and Upsloping (slope=0) have slightly lower predicted risks (~0.52–0.53).

This suggests the model sees downsloping ST segments as a stronger indicator of heart disease risk.

#thal
The model predicts highest risk (~0.59–0.60) for categories 2 and slightly lower for 0/1.

The lowest risk (~0.45) is predicted for category 3.

This indicates the model views thal=2 (likely a reversible defect) as the strongest risk indicator, while thal=3 is the least associated with heart disease.

#oldpeak
The average predicted risk starts higher (~0.6) when oldpeak is near 0, then declines as oldpeak increases up to ~3, after which it stabilizes.

This trend suggests the model predicts lower heart disease probability as ST depression increases, which may reflect dataset-specific patterns or feature interaction.

The ICE curves show high variability, with some predictions dropping sharply and others staying high, indicating heterogeneous effects across individuals.


# PDP/ICE for KNN

In [ ]:
top_7_features = ['cp', 'chol', 'age', 'sex', 'thal', 'exang', 'thalach']

#For KNN
plot_pdp_ice_all_numeric_safe(results["KNN"]["pipeline"],
                              X_train_raw,
                              "KNN",
                              top_7_features)

# sex
The average predicted probability of heart disease is higher (~0.65) for individuals females, compared to ~0.50 for males.

The KNN model is using sex as a strong differentiator in its predictions, with a noticeable gap in risk between the two groups.

# exang
The average predicted probability of heart disease is higher (~0.59) for people with no exercise-induced angina and lower (~0.49) for presence of angina.

The KNN model sees no angina as linked to higher predicted risk.

# slope
The model predicts a clear increase in risk as the slope category shifts toward more abnormal ST segments

# thal
The average predicted probability of heart disease is highest (~0.65) for thal=1, slightly lower for thal=0, and then decreases for thal=2 and most noticeably for thal=3 (~0.50).

The KNN model treats thal as a risk-differentiating factor, with thal=1 (likely fixed defect) linked to the greatest predicted risk, while thal=3 corresponds to the lowest.

These differences suggest the model views certain thalassemia categories (particularly thal=1) as strong indicators of higher heart disease risk, likely reflecting patterns in the training dataset rather than a direct causal effect.

# oldpeak
The average predicted probability of heart disease (dashed line) is highest (~0.6) when oldpeak is near 0 and declines steadily as oldpeak increases, dropping below 0.3 around 6.

This trend implies the KNN model predicts lower heart disease risk at higher ST depression values, which may reflect dataset-specific patterns or interactions with other features rather than a direct causal link.

The ICE curves show significant variability — some patients’ predicted risks remain high while others drop sharply, suggesting strong interactions between oldpeak and other factors in the model.

# PDP/ICE for XGBoost

In [ ]:
top_7_features =['cp', 'chol', 'age', 'sex', 'thal', 'ca', 'thalach']

#For XGBoost
plot_pdp_ice_all_numeric_safe(results["XGBoost"]["pipeline"],
                              X_train_raw,
                              "XGBoost",
                              top_7_features)

NameError: name 'plot_pdp_ice_all_numeric_safe' is not defined

# PDP/ICE for Multi-Layer Perpetualtion

In [ ]:
top_7_features = ['cp', 'chol', 'age', 'sex', 'ca', 'slope', 'thalach']


#For KNN
plot_pdp_ice_all_numeric_safe(results["MLP"]["pipeline"],
                              X_train_raw,
                              "MLP",
                              top_7_features)

# Counterfactuals

In [ ]:
def generate_counterfactuals_for_model(model_name, sample_idx=43, total_CFs=3):

    if model_name not in results:
        raise ValueError(f"Model '{model_name}' not found. Available: {list(results.keys())}")

    #Combine train features and target
    data = X_train_raw.copy()
    data['target'] = y_train.values

    #Define continuous and categorical features
    continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
    categorical_features = [col for col in X_train_raw.columns if col not in continuous_features]

    #  Create DiCE data interface
    dice_data = dice_ml.Data(
        dataframe=data,
        continuous_features=continuous_features,
        outcome_name='target'
    )

    #Wrap pipeline for DiCE
    pipeline = results[model_name]["pipeline"]
    dice_model = dice_ml.Model(model=pipeline, backend="sklearn")

    #create the DiCE explainer
    exp = Dice(data_interface=dice_data, model_interface=dice_model, method="genetic")

    #Select the patient to generate counterfactuals for
    sample = X_test_raw.iloc[[sample_idx]]
    print(f"\nOriginal model prediction ({model_name}):", pipeline.predict(sample))

    # Generate counterfactuals
    counterfactuals = exp.generate_counterfactuals(
        sample,
        total_CFs=total_CFs,
        desired_class="opposite",  #flip 0→1 or 1→0
    )


    #Show results
    return counterfactuals.visualize_as_dataframe()

#Generate for each model
for model in ["Random Forest", "KNN", "XGBoost", "MLP"]:
    print(f"\n=== Counterfactuals for {model} ===")
    display(generate_counterfactuals_for_model(model, sample_idx=43, total_CFs=3))



=== Counterfactuals for Random Forest ===


NameError: name 'results' is not defined

**Random Forest**
1. The model is very sensitive to age, chol, and trestbps for this patient.

2. By reducing age and slightly modifying blood pressure and cholesterol, the predicted risk drops enough to flip from 1 (disease) to 0 (no disease).

3. All other features (sex, cp, thalach, thal, etc.) remain unchanged because:

* We locked them (not allowed to vary), or

* The model doesn’t rely on them as heavily for this patient.

# Surrogate Model (Decision Tree)

In [ ]:
#Change model here
blackbox_model_name = "MLP" #Random Forest, KNN, XGBoost, MLP

#get model and its pipeline
blackbox_pipeline = results[blackbox_model_name]["pipeline"]
blackbox_model = blackbox_pipeline.named_steps["classifier"]
preprocessor = blackbox_pipeline.named_steps["preprocessor"]

#Preprocess the training data
X_train_processed = preprocessor.transform(X_train_raw)

#get the predicted class labels
y_pred_blackbox_classes = blackbox_model.predict(X_train_processed)

# Train surrogate on processed training data and b-b predicted classes
surrogate = DecisionTreeClassifier(max_depth=4, random_state=42)  # limit depth for simplicity
surrogate.fit(X_train_processed, y_pred_blackbox_classes)


#Print the tree's rules
#feature names after preprocessing
feature_names_processed = preprocessor.get_feature_names_out()
tree_rules = export_text(surrogate, feature_names=list(feature_names_processed))
print("Surrogate Decision Tree Rules:")
print(tree_rules)

#Visualize tree
plt.figure(figsize=(20, 10)) # Increased figure size for better visibility
plot_tree(surrogate, feature_names=feature_names_processed, class_names=["No Disease", "Disease"], filled=True, fontsize=10)
plt.title(f"Surrogate Decision Tree for {blackbox_model_name} (max_depth=4)")
plt.show()


# Aggregation of Local -> Global

In [ ]:
############################### LIME #####################################
k = 61  #Number of instances to analyze
lime_feature_counts = {}

#Extract trained pipeline parts
rf_pipeline = results["XGBoost"]["pipeline"]
preprocessor = rf_pipeline.named_steps["preprocessor"]
rf_model = rf_pipeline.named_steps["classifier"]

#Preprocess
X_train_processed = preprocessor.transform(X_train_raw)

#LIME explainer
explainer = LimeTabularExplainer(
    training_data=X_train_processed,
    feature_names=preprocessor.get_feature_names_out(),
    class_names=['No Disease', 'Heart Disease'],
    mode='classification',
    discretize_continuous=True
)

def rf_predict_proba(x):
    return rf_model.predict_proba(x)

#Loop through k instances in test set
for i in range(k):
    X_one_raw = X_test_raw.iloc[[i]]
    X_one_transformed = preprocessor.transform(X_one_raw)

    #Explain one instance
    lime_exp = explainer.explain_instance(
        data_row=X_one_transformed[0],
        predict_fn=rf_predict_proba,
        num_features=7
    )

    #Collect feature importance
    for feature, weight in lime_exp.as_list():
        if feature not in lime_feature_counts:
            lime_feature_counts[feature] = 0
        lime_feature_counts[feature] += 1

#Sort and print most frequently important features
sorted_features = sorted(lime_feature_counts.items(), key=lambda x: x[1], reverse=True)

print(f"\nTop features from LIME - across {k} instances:")
for feature, count in sorted_features:
    print(f"{feature}: {count} times")


In [ ]:
################################## Counterfactuals #####################################
cf_feature_counts = {}

#trained MLmodel pipeline
rf_pipeline = results["Random Forest"]["pipeline"]
preprocessor = rf_pipeline.named_steps["preprocessor"]
rf_model = rf_pipeline.named_steps["classifier"]

#Prepare data for DiCE
data = X_train_raw.copy()
data['target'] = y_train.values

continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = [col for col in X_train_raw.columns if col not in continuous_features]

dice_data = dice_ml.Data(
    dataframe=data,
    continuous_features=continuous_features,
    outcome_name='target'
)

dice_model = dice_ml.Model(model=rf_pipeline, backend="sklearn") # Use the full pipeline-

#Create the DiCE explainer
dice_explainer = Dice(data_interface=dice_data, model_interface=dice_model, method="genetic")


for i in range(61):
    # Get the instance
    query_instance = X_test_raw.iloc[i:i+1]

    try:
        #Generate counterfactuals
        cf = dice_explainer.generate_counterfactuals(
          query_instance,
          total_CFs=1,
          desired_class="opposite"
          )

        #FIX ERROR
        #did CF generated successfully
        if cf and cf.cf_examples_list and len(cf.cf_examples_list) > 0 and cf.cf_examples_list[0].final_cfs_df is not None:
            cf_df = cf.cf_examples_list[0].final_cfs_df

            #Compare instance and counterfactual
            for col in query_instance.columns:
                original = query_instance[col].values[0]
                counterfactual = cf_df[col].values[0]

                if original != counterfactual:
                    #Count the changed feature
                    if col not in cf_feature_counts:
                        cf_feature_counts[col] = 0
                    cf_feature_counts[col] += 1
        else:
            print(f"Could not generate counterfactuals for instance {i}")

    except Exception as e:
        print(f"Error generating counterfactuals for instance {i}: {e}")


#Sort and display
sorted_cf_features = sorted(cf_feature_counts.items(), key=lambda x: x[1], reverse=True)

print("\nTop features changed in Counterfactuals across 61 instances:")
for feature, count in sorted_cf_features:
    print(f"{feature}: {count} times")

# RBO

In [ ]:
def rbo_truncated(list_a, list_b, p=0.9, k=None):
    #truncated RBO calculation up to depth k
    if k is None:
        k = min(len(list_a), len(list_b))
    seen_a, seen_b = set(), set()
    s = 0.0
    for d in range(1, k + 1):
        seen_a.add(list_a[d-1])
        seen_b.add(list_b[d-1])
        overlap_d = len(seen_a & seen_b)  #overlap at depth d
        A_d = overlap_d / d                #agreement ratio
        s += (p ** (d - 1)) * A_d          #weighted sum
    return s, A_d


def rbo_full_from_truncated(raw_sum, A_k, p=0.9, k=7):
    #Complete RBO with infinite tail correction
    return (1 - p) * (raw_sum + (A_k * (p ** k)) / (1 - p))


#Edit as needed
expert_top7 = ["num__cp","num__chol","num__trestbps","num__fbs","num__exang","num__age","num__sex"]
model_top7  = ["num__cp","num__chol","num__age","num__sex","num__ca","num__slope","num__thalach"]

p = 0.9

features = [f for f in df.columns if f != "target"] #Exclude target column

#truncated raw sum + last agreement
raw_sum, A_k = rbo_truncated(expert_top7, model_top7, p, k=7)

#compute RBO, normalized
full_rbo = rbo_full_from_truncated(raw_sum, A_k, p, k=7)

print("Truncated RBO:", round((1 - p) * raw_sum, 3))
print("Full RBO:", round(full_rbo, 3))



# Vector Similarity

In [ ]:
# Cumulative Sum values for each model
rf_data = {
    "LIME_RF": [0, 0, 0, 0, 0.013122, 0.0426465, 0.0654225],
    "SHAP_RF": [0.1, 0.145, 0.172, 0.190255, 0.203347, 0.211388, 0.220781],
    "PFI_RF": [0, 0, 0, 0, 0, 0.009842, 0.02502],
    "PDP_RF": [0.1, 0.19, 0.244, 0.2804, 0.306969, 0.346065, 0.376472],
    "Counterfactuals_RF": [0, 0, 0.027, 0.06345, 0.089694, 0.12906, 0.159428],
    "Surrogate_RF": [0.1, 0.145, 0.172, 0.190225, 0.203347, 0.22303, 0.245806]
}

knn_data = {
    "LIME_KNN": [0, 0, 0, 0, 0.026244, 0.045927, 0.061111],
    "SHAP_KNN": [0, 0, 0, 0.018225, 0.044469, 0.064152, 0.08693],
    "PFI_KNN": [0, 0, 0, 0, 0, 0.053144, 0.053144],
    "PDP_KNN": [0.1, 0.19, 0.244, 0.2804, 0.30669, 0.33621, 0.366616],
    "Counterfactuals_KNN": [0, 0.045, 0.072, 0.10845, 0.13469, 0.17406, 0.20442],
    "Surrogate_KNN": [0.1, 0.145, 0.172, 0.190225, 0.216469, 0.236152, 0.258928]
}

xgb_data = {
    "LIME_XGB": [0, 0, 0, 0.018225, 0.031347, 0.05103, 0.073806],
    "SHAP_XGB": [0.1, 0.145, 0.172, 0.190225, 0.216469, 0.236152, 0.258951],
    "PFI_XGB": [0, 0, 0, 0, 0.00984, 0.02502, 0.02502],
    "PDP_XGB": [0.1, 0.19, 0.244, 0.2804, 0.30669, 0.33621, 0.36661],
    "Counterfactuals_XGB": [0, 0.045, 0.072, 0.10845, 0.134694, 0.17406, 0.204428],
    "Surrogate_XGB": [0.1, 0.145, 0.172, 0.190225, 0.203347, 0.213188, 0.228373]
}

mlp_data = {
    "LIME_MLP": [0, 0, 0, 0.018225, 0.031347, 0.05103, 0.073806],
    "SHAP_MLP": [0.1, 0.145, 0.172, 0.190225, 0.216469, 0.236152, 0.258951],
    "PFI_MLP": [0, 0, 0, 0, 0.00984, 0.02502, 0.02502],
    "PDP_MLP": [0.1, 0.19, 0.244, 0.2804, 0.30669, 0.33621, 0.36661],
    "Counterfactuals_MLP": [0, 0.045, 0.072, 0.10845, 0.134694, 0.17406, 0.204428],
    "Surrogate_MLP": [0.1, 0.145, 0.172, 0.190225, 0.203347, 0.213188, 0.228373]
}

# Function to compute cosine similarity and Spearman correlation
def compute_similarity(df):

    #Cosine
    cosine_sim = cosine_similarity(df.T)
    cosine_df = pd.DataFrame(cosine_sim, index=df.columns, columns=df.columns)

    #Spearman
    methods = df.columns
    spearman_matrix = pd.DataFrame(index=methods, columns=methods, dtype=float)
    for i in methods:
        for j in methods:
            rho, _ = spearmanr(df[i], df[j])
            spearman_matrix.loc[i, j] = rho

    return cosine_df, spearman_matrix

# Run for all models
with pd.ExcelWriter("Vector_Similarity_All_Models.xlsx") as writer: #Save results to Excel
    for model_name, model_data in {
        "RF": rf_data,
        "KNN": knn_data,
        "XGB": xgb_data,
        "MLP": mlp_data
    }.items():
        df = pd.DataFrame(model_data)
        cosine_df, spearman_matrix = compute_similarity(df)

        #2 separate excel sheets
        cosine_df.to_excel(writer, sheet_name=f"{model_name}_Cosine")
        spearman_matrix.to_excel(writer, sheet_name=f"{model_name}_Spearman")

print("Similarity matrices saved")


# Time/Space

In [ ]:
performance_results = []

def lime_explainer(pipeline, X):
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    explainer = LimeTabularExplainer(
        training_data=preprocessor.transform(X_train_raw),   #background from training set
        feature_names=preprocessor.get_feature_names_out(),  #names after preprocessing
        class_names=['No Disease', 'Heart Disease'],
        mode='classification'
    )
    #explain one instance
    _ = explainer.explain_instance(
        data_row=preprocessor.transform(X.iloc[[0]])[0],
        predict_fn=model.predict_proba,
        num_features=7
    )

def shap_explainer(pipeline, X):
    model = pipeline.named_steps["classifier"]
    preprocessor = pipeline.named_steps["preprocessor"]
    X_proc = preprocessor.transform(X)
    explainer = shap.Explainer(model.predict, X_proc)
    # run on first 10 samples
    _ = explainer(X_proc[:10])

def pfi_explainer(pipeline, X, y):
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    _ = permutation_importance(model, X_proc, y, n_repeats=5)

def pdp_explainer(pipeline, X):
    #average effect of features (first 5 features only)
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    features = list(range(min(5, X_proc.shape[1]))) #first 5 features only
    PartialDependenceDisplay.from_estimator(model, X_proc, features)

def counterfactual_explainer(pipeline, X):
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    d = dice_ml.Data(
        dataframe=pd.concat([X_train_raw, y_train], axis=1),
        continuous_features=X_train_raw.columns.tolist(),
        outcome_name=y_train.name
    )
    m = dice_ml.Model(model=model, backend="sklearn")
    exp = dice_ml.Dice(d, m)
    _ = exp.generate_counterfactuals(X_train_raw.iloc[[0]], total_CFs=1)

def surrogate_explainer(pipeline, X):
    from sklearn.tree import DecisionTreeClassifier
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    y_pred = model.predict(X_proc)
    surrogate = DecisionTreeClassifier(max_depth=3)
    surrogate.fit(X_proc, y_pred)

#Connect names to functions
xai_methods = {
    "LIME": lambda p, X: lime_explainer(p, X),
    "SHAP": lambda p, X: shap_explainer(p, X),
    "PFI": lambda p, X: pfi_explainer(p, X, y_test),
    "PDP": lambda p, X: pdp_explainer(p, X),
    "Counterfactuals": lambda p, X: counterfactual_explainer(p, X),
    "Surrogate": lambda p, X: surrogate_explainer(p, X)
}

  #run performance tests and measure time/memory for each model and method combo
for model_name, model_data in results.items():
    pipeline = model_data["pipeline"]
    for method_name, method_func in xai_methods.items():
        print(f"Running {method_name} for {model_name}...")
        start_time = time.perf_counter()     #start timer
        tracemalloc.start()                  #start memory tracking
        try:
            method_func(pipeline, X_test_raw)   #run explainer
        except Exception as e:
            print(f" ERROR with {method_name} for {model_name}: {e}")
        current, peak = tracemalloc.get_traced_memory()  #peak memory usage
        tracemalloc.stop()
        end_time = time.perf_counter()     #end timer

        #Store results
        performance_results.append({
            "Model": model_name,
            "Method": method_name,
            "Time_sec": round(end_time - start_time, 4),
            "Peak_MB": round(peak / (1024 * 1024), 4)
        })


#Save results in Excel
df_perf = pd.DataFrame(performance_results)
df_perf.to_excel("Performance_Results.xlsx", index=False)
print("Performance metrics saved")


In [ ]:
# Time plot
plt.figure(figsize=(10,6))

sns.barplot(        #Barplot for each method across models
    data=df_perf,
    x="Method",
    y="Time_sec",
    hue="Model",
    palette="Set2"
)
plt.title("XAI Runtime Performance")
plt.ylabel("Execution Time (seconds)")
plt.xlabel("XAI Method")
plt.xticks(rotation=30)
plt.legend(title="Model")
plt.tight_layout()
plt.show()

# --- Memory plot ---
plt.figure(figsize=(10,6))
sns.barplot(
    data=df_perf,
    x="Method",
    y="Peak_MB",
    hue="Model",
    palette="Set2"
)
plt.title("XAI Memory Usage Performance")
plt.ylabel("Peak Memory (MB)")
plt.xlabel("XAI Method")
plt.xticks(rotation=30)
plt.legend(title="Model")
plt.tight_layout()
plt.show()


# Stability test

In [ ]:
def lime_stability(pipeline, X):
    #get feature weights for one instance
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    explainer = LimeTabularExplainer(
        training_data=preprocessor.transform(X_train_raw),
        feature_names=preprocessor.get_feature_names_out(),
        class_names=['No Disease', 'Heart Disease'],
        mode='classification'
    )
    exp = explainer.explain_instance(
        data_row=preprocessor.transform(X.iloc[[0]])[0],
        predict_fn=model.predict_proba,
        num_features=7
    )
    #Convert explanation to a vector aligned with feature names
    weights = dict(exp.as_list())
    return np.array([weights.get(f, 0) for f in preprocessor.get_feature_names_out()])


def shap_stability(pipeline, X):
    #average absolute shap values across 10 samples
    model = pipeline.named_steps["classifier"]
    preprocessor = pipeline.named_steps["preprocessor"]
    X_proc = preprocessor.transform(X)
    explainer = shap.Explainer(model.predict, X_proc)
    shap_values = explainer(X_proc[:10])
    return np.mean(np.abs(shap_values.values), axis=0)


def pfi_stability(pipeline, X, y):
    #mean score across runs
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    r = permutation_importance(model, X_proc, y, n_repeats=5)
    return r.importances_mean


def pdp_stability(pipeline, X):
    #use the variance of PDP curve as a proxy for feature stability
    from sklearn.inspection import partial_dependence
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    features = list(range(min(5, X_proc.shape[1])))
    pdp_vals = []
    for f in features:
        pdp_result = partial_dependence(model, X_proc, [f])
        pdp_vals.append(np.var(pdp_result.average))
    return np.array(pdp_vals)


def counterfactuals_stability(pipeline, X):
    #mark which features change between instance and CF
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    d = dice_ml.Data(
        dataframe=pd.concat([X_train_raw, y_train], axis=1),
        continuous_features=X_train_raw.columns.tolist(),
        outcome_name=y_train.name
    )
    m = dice_ml.Model(model=model, backend="sklearn")
    exp = dice_ml.Dice(d, m)
    cf = exp.generate_counterfactuals(X_train_raw.iloc[[0]], total_CFs=1)
    changed = cf.cf_examples_list[0].final_cfs_df.iloc[0] != X_train_raw.iloc[0]
    return changed.astype(int).values


def surrogate_stability(pipeline, X):
    #fit a decision tree and return feature importances
    from sklearn.tree import DecisionTreeClassifier
    preprocessor = pipeline.named_steps["preprocessor"]
    model = pipeline.named_steps["classifier"]
    X_proc = preprocessor.transform(X)
    y_pred = model.predict(X_proc)
    surrogate = DecisionTreeClassifier(max_depth=3)
    surrogate.fit(X_proc, y_pred)
    return surrogate.feature_importances_

#Connect names to functions
xai_methods_stability = {
    "LIME": lambda p, X: lime_stability(p, X),
    "SHAP": lambda p, X: shap_stability(p, X),
    "PFI": lambda p, X: pfi_stability(p, X, y_test),
    "PDP": lambda p, X: pdp_stability(p, X),
    "Counterfactuals": lambda p, X: counterfactuals_stability(p, X),
    "Surrogate": lambda p, X: surrogate_stability(p, X)
}


In [ ]:
stability_results = []

repeats = 5  # number of times to repeat explanation
#Loop thru each model and XAI method
for model_name, model_data in results.items():
    pipeline = model_data["pipeline"]
    for method_name, method_func in xai_methods_stability.items():
        all_runs = []

        for _ in range(repeats):               #repeat explanations multiple times
            try:
                vec = method_func(pipeline, X_test_raw) #numeric vector from method
                all_runs.append(vec)
            except Exception as e:
                print(f"{method_name} failed for {model_name}: {e}")
                break
        if all_runs:
            arr = np.vstack(all_runs)       #stack results from all runs
            mean_val = np.mean(arr)         #mean stability score
            std_val = np.std(arr)           #std dev of stability score
            stability_results.append({
                "Model": model_name,
                "Method": method_name,
                "Mean": mean_val,
                "StdDev": std_val
            })
#Save in excel
df_stability = pd.DataFrame(stability_results)
df_stability.to_excel("Stability_Results.xlsx", index=False)
print("Stability results saved")


NameError: name 'results' is not defined

In [ ]:
#VISUALIZATION OF STABILITY
plt.figure(figsize=(10,6))
sns.barplot(
    data=df_stability,
    x="Method",
    y="Mean",
    hue="Model",
    palette="Set2",
    capsize=0.1,
    errwidth=1,
    ci=None
)

#Add error bars manually from StdDev
for i, row in df_stability.iterrows():
    plt.errorbar(
        x=i % len(df_stability['Method'].unique()),  #position within group
        y=row['Mean'],
        yerr=row['StdDev'],
        fmt='none',
        c='black',
        capsize=5
    )

plt.title("Stability Robustness of XAI Methods (Mean ± Std Dev)")
plt.ylabel("Stability Score")
plt.xlabel("XAI Method")
plt.legend(title="Model")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

# T-SNE

# Fidility

In [ ]:
#how well a explanation matches the models predictions

def fidelity_lime(pipeline, X_train, X_test, y_test, sample_size=10):
    #Fidelity for LIME using R² of local surrogate

    #keep original feature names
    X_train_df = pd.DataFrame(X_train, columns=X.columns)
    X_test_df  = pd.DataFrame(X_test, columns=X.columns)

    explainer = LimeTabularExplainer(
        training_data=X_train_df.values,
        feature_names=X_train_df.columns.tolist(),
        class_names=["No Disease", "Disease"],
        mode="classification"
    )

    local_scores = []
    #explain a few test samples
    for i in range(min(sample_size, len(X_test_df))):
        exp = explainer.explain_instance(
            data_row=X_test_df.iloc[i].values,
            predict_fn=lambda x: pipeline.predict_proba(pd.DataFrame(x, columns=X.columns))
        )
        local_scores.append(exp.score)  #R2 score for the local surrogate

    return np.mean(local_scores)



def fidelity_shap(pipeline, X_train, X_test, sample_size=50):
    #correlation between true probs and SHAP reconstruction
    preproc = pipeline.named_steps['preprocessor']
    model   = pipeline.named_steps['classifier']

    #preprocess data
    X_train_proc = preproc.fit_transform(X_train)
    X_test_proc  = preproc.transform(X_test)

    #background and sample
    background = shap.sample(X_train_proc, 50, random_state=42)
    X_sample   = X_test_proc[:sample_size]

    #only predict prob for class 1 (Heart disease)
    f = lambda x: model.predict_proba(x)[:, 1]

    explainer = shap.KernelExplainer(f, background)
    shap_values = explainer.shap_values(X_sample)

    expected_value = explainer.expected_value
    reconstructed = expected_value + shap_values.sum(axis=1)

    preds = f(X_sample)

    #correlation between SHAP reconstruction and model probs
    return float(np.corrcoef(preds, reconstructed)[0, 1])



def fidelity_counterfactual(pipeline, X_train, X_test, y_train):
    #how often a counterfactual can be generated.

    df_train = X_train.copy()
    df_train['target'] = y_train.values

    data = dice_ml.Data(
        dataframe=df_train,
        continuous_features=X_train.columns.tolist(),
        outcome_name="target"
    )
    model = dice_ml.Model(model=pipeline, backend="sklearn")
    exp = Dice(data, model)

    success = 0
    total = min(20, len(X_test))  #keep runtime reasonable
    for i in range(total):
        e1 = exp.generate_counterfactuals(X_test.iloc[[i]], total_CFs=1, desired_class="opposite")
        if e1.cf_examples_list[0].final_cfs_df is not None:
            success += 1
    return success / total



def fidelity_surrogate_tree(pipeline, X_test):
    # how accurate the tree mimics

    preds = pipeline.predict(X_test)
    surrogate = DecisionTreeClassifier(max_depth=5, random_state=42)
    surrogate.fit(X_test, preds)
    return surrogate.score(X_test, preds)

def fidelity_pfi(pipeline, X_test, y_test):
    #mean permutation importance

    result = permutation_importance(pipeline, X_test, y_test, n_repeats=5, random_state=42)
    return np.mean(result.importances_mean)


def fidelity_pdp(pipeline, X_test):
    #correlation of predictions with PDP averages

    preds = pipeline.predict_proba(X_test)[:, 1]
    pdp_values = []
    for col in X_test.columns:
        display = PartialDependenceDisplay.from_estimator(pipeline, X_test, [col])
        avg_effect = np.mean(display.deciles[0])
        pdp_values.append(avg_effect)
        plt.close('all')  #avoid too many figures

    if len(pdp_values) > 0:
        return np.corrcoef(
            preds,
            np.tile(pdp_values, len(preds)//len(pdp_values)+1)[:len(preds)]
        )[0, 1]
    else:
        return np.nan



#Fidelity comparison across models

fidelity_results = []

#make sure data are DataFrames
X_train_raw = pd.DataFrame(X_train_raw, columns=X.columns)
X_test_raw  = pd.DataFrame(X_test_raw, columns=X.columns)

for name, metrics in results.items():
    pipeline = metrics["pipeline"]
    row = {"Model": name}

    try:
        row["LIME"] = fidelity_lime(pipeline, X_train_raw, X_test_raw, y_test)
    except Exception as e:
        print(f"LIME failed for {name}: {e}")
        row["LIME"] = np.nan

    try:
        row["SHAP"] = fidelity_shap(pipeline, X_train_raw, X_test_raw)
    except Exception as e:
        print(f"SHAP failed for {name}: {e}")
        row["SHAP"] = np.nan

    try:
        row["Counterfactual"] = fidelity_counterfactual(pipeline, X_train_raw, X_test_raw, y_train)
    except Exception as e:
        print(f"Counterfactuals failed for {name}: {e}")
        row["Counterfactual"] = np.nan

    try:
        row["Surrogate Tree"] = fidelity_surrogate_tree(pipeline, X_test_raw)
    except Exception as e:
        print(f"Surrogate tree failed for {name}: {e}")
        row["Surrogate Tree"] = np.nan

    try:
        row["PFI"] = fidelity_pfi(pipeline, X_test_raw, y_test)
    except Exception as e:
        print(f"PFI failed for {name}: {e}")
        row["PFI"] = np.nan

    try:
        row["PDP"] = fidelity_pdp(pipeline, X_test_raw)
    except Exception as e:
        print(f"PDP failed for {name}: {e}")
        row["PDP"] = np.nan

    fidelity_results.append(row)

#DataFrame for easy comparison
fidelity_df = pd.DataFrame(fidelity_results)
fidelity_df = fidelity_df.round(3)

print("\nFidelity Comparison Table:")
print(fidelity_df.to_markdown(index=False))


In [ ]:
#wide table to long format seaborn barplot
fidelity_long = fidelity_df.melt(id_vars="Model", var_name="Method", value_name="Fidelity")

# Barplot compares fidelity scores of each XAI method across models
plt.figure(figsize=(10,6))
sns.barplot(
    data=fidelity_long,
    x="Method",
    y="Fidelity",
    hue="Model",
    palette="Set2"
)

plt.title("Explainability Fidelity (per XAI Method)")
plt.ylabel("Fidelity Score")
plt.xlabel("XAI Method")
plt.xticks(rotation=30)
plt.legend(title="Model")
plt.tight_layout()
plt.show()
